In [1]:
# imports
import pandas as pd
import numpy as np
import sys
import argparse
from NFL_Dataloader import GameSummary,NFLAPI_Processor
from scrapers import ApiGameLinks
from NFL_RefMaps import TableColumns
from NFL_Metrics import SkillPoints
from sqlalchemy import create_engine

In [4]:
season = 2019
weeks = np.arange(1,11).tolist()
weeks
week=1

In [5]:
# scrape play by play from API for week
api_games = ApiGameLinks(season,week)
gameids = api_games.get_gameids()

In [6]:
pbp_df = pd.DataFrame(columns=TableColumns().nflapi['pbp_cols'])
for game in gameids:
    pbp = NFLAPI_Processor(game).process_nflapi()
    pbp_df = pd.concat([pbp_df,pbp],axis=0)

# pbp_df = pd.concat([pbp_df,pbp],verify_integrity=True)
# pbp_df.head()

In [7]:
pbp_df

,play_id,game_id,home_team,away_team,posteam,yardline_100,game_date,quarter_end,drive,sp,...,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,replay_or_challenge,replay_or_challenge_result,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,35,2019090500,CHI,GB,CHI,65,09-05-2019,0,1,0,...,N/A,N/A,GB,0.0,0,N/A,0,0,0,0
1,50,2019090500,CHI,GB,GB,75,09-05-2019,0,1,0,...,N/A,N/A,N/A,0.0,0,N/A,0,0,0,0
2,71,2019090500,CHI,GB,GB,75,09-05-2019,0,1,0,...,N/A,N/A,N/A,0.0,0,N/A,0,0,0,0
3,95,2019090500,CHI,GB,GB,75,09-05-2019,0,1,0,...,N/A,N/A,N/A,0.0,0,N/A,0,0,0,0
4,125,2019090500,CHI,GB,GB,85,09-05-2019,0,1,0,...,N/A,N/A,N/A,0.0,0,N/A,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,3628,2019090901,OAK,DEN,,0,09-09-2019,0,16,0,...,N/A,N/A,N/A,0.0,0,N/A,0,0,0,0
155,3645,2019090901,OAK,DEN,OAK,73,09-09-2019,0,16,0,...,N/A,N/A,N/A,0.0,0,N/A,0,0,0,0
156,3669,2019090901,OAK,DEN,OAK,63,09-09-2019,0,16,0,...,N/A,N/A,N/A,0.0,0,N/A,0,0,0,0
157,3690,2019090901,OAK,DEN,OAK,50,09-09-2019,0,16,0,...,N/A,N/A,N/A,0.0,0,N/A,0,0,0,0


In [12]:
# initialize database sql engines
nfldb_engine = create_engine('mysql+pymysql://root:@localhost:3306/main_stats')
main_engine = nfldb_engine.connect()

In [13]:
sql = 'select * from nfl_pbp limit 2'

In [14]:
df = pd.read_sql(sql,con=main_engine)
diff = set(df.columns)-set(pbp_df.columns)
diff

set()

In [15]:
pbp_df.to_sql('nfl_pbp', con=main_engine, if_exists='append',index=False)

In [16]:
sql = 'select * from nfl_game_summary limit 2'
df = pd.read_sql(sql,con=main_engine)
df.columns

Index(['game_id', 'schedule_date', 'schedule_season', 'schedule_week',
       'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line',
       'home_favorite', 'spread_result', 'OU_result', 'pfr_gamelinks',
       'season', 'week', 'home_abbrev', 'home_rush_yds', 'home_rush_att',
       'home_rush_tds', 'home_shortpass_yds', 'home_shortpass_att',
       'home_shortpass_completions', 'home_shortpass_tds', 'home_deeppass_yds',
       'home_deeppass_att', 'home_deeppass_completions', 'home_deeppass_tds',
       'home_sacked', 'home_interceptions', 'away_abbrev', 'away_rush_yds',
       'away_rush_att', 'away_rush_tds', 'away_shortpass_yds',
       'away_shortpass_att', 'away_shortpass_completions',
       'away_shortpass_tds', 'away_deeppass_yds', 'away_deeppass_att',
       'away_deeppass_completions', 'away_deeppass_tds', 'away_sacked',
       'away_interceptions'],
      dtype='object')